<a href="https://colab.research.google.com/github/tarakantaacharya/CovidPneumonia-Case-study/blob/main/Copy_of_OriginalWebapplication1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask keras tensorflow pyngrok

In [2]:
!pip install pyngrok

In [ ]:
!ls

sample_data


In [3]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyngrok import ngrok

In [6]:
!ngrok authtoken 2jEsPzf639Wdp2EFpeDqjRkvamr_7CW5emBByF7rE3WJ6efMN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
import shutil

# Define source and destination paths
source = '/content/drive/MyDrive/XrayDetection/templates'
destination = '/content/templates'

# Move the file
shutil.move(source, destination)


'/content/templates'

In [10]:
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import cv2
import base64
from pyngrok import ngrok

app = Flask(__name__)
model = load_model('/content/drive/MyDrive/XrayDetection/newpathmodel3.keras')

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def preprocess_image(img_path, target_size=(256, 256)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

def classify_image(img_path, model):
    input_image = preprocess_image(img_path)
    predictions = model.predict(input_image)
    predicted_class_index = np.argmax(predictions, axis=1)
    class_names = ['COVID-19/कोविड/కోవిడ్', 'NORMAL/सामान्य/సాధారణ', 'PNEUMONIA/न्यूमोनिया/న్యుమోనియా']
    predicted_class_label = class_names[predicted_class_index[0]]
    return predicted_class_label, predictions[0]

@app.route('/')
def home():
    return render_template('index7.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    global model

    if 'file' not in request.files:
        return jsonify({'result': "No file part"})

    file = request.files['file']

    if file.filename == '':
        return jsonify({'result': "No selected file"})

    if file and allowed_file(file.filename):
        try:
            # Save the file temporarily
            filename = secure_filename(file.filename)
            file_path = os.path.join('/tmp', filename)
            file.save(file_path)

            # Classify the image
            predicted_class_label, probabilities = classify_image(file_path, model)

            # Load the original image for display
            original_img = cv2.imread(file_path)
            original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

            # Remove the temporary file
            os.remove(file_path)

            # Prepare data to send to the template
            img_str = cv2.imencode('.jpg', original_img)[1].tobytes()
            img_base64 = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')

            result_text = f"COVID-19/कोविड/కోవిడ్: {probabilities[0]:.2f}, NORMAL/सामान्य/సాధారణ: {probabilities[1]:.2f}, PNEUMONIA/न्यूमोनिया/న్యుమోనియా: {probabilities[2]:.2f}"

            return render_template('index2.html', prediction=predicted_class_label, probabilities=result_text, image=img_base64)

        except Exception as e:
            return jsonify({'result': f"Error processing file: {str(e)}"})

    else:
        return jsonify({'result': "File type not allowed"})

if __name__ == '__main__':
    ngrok.set_auth_token("2jEsPzf639Wdp2EFpeDqjRkvamr_7CW5emBByF7rE3WJ6efMN")
    public_url = ngrok.connect(5000)
    print(f"Ngrok Tunnel: {public_url}")

    app.run(host='0.0.0.0', port=5000)


Ngrok Tunnel: NgrokTunnel: "https://4365-34-74-59-254.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:43:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:43:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:43:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:43:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:44:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:44:18] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step


INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:44:39] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:46:35] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:47:43] "POST /upload HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:48:24] "POST /upload HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


INFO:werkzeug:127.0.0.1 - - [06/Aug/2024 11:49:27] "POST /upload HTTP/1.1" 200 -


In [11]:
import shutil

# Define source and destination paths
destination = '/content/drive/MyDrive/XrayDetection/templates'
source = '/content/templates'


# Move the file
shutil.move(source, destination)

'/content/drive/MyDrive/XrayDetection/templates'